In [6]:
# don't know why but need an update
!sudo apt update
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.1
!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
77 packages can be upgraded. Run 'ap

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

import findspark
findspark.init()

In [8]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"USERNAME","key":"KAGGLE_KEY"}'}

In [10]:
# run this code in order to provide the API and connect to Kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# this command is used to access "census/2013..." and download the zip file
!kaggle datasets download -d census/2013-american-community-survey

100% 916M/916M [00:10<00:00, 104MB/s]
100% 916M/916M [00:10<00:00, 88.8MB/s]


In [11]:
# now that everything is set, we need to unzip the file and load in the notebook 
import zipfile
zip_ref = zipfile.ZipFile('2013-american-community-survey.zip', 'r')
zip_ref.extractall('files')
zip_ref.close()

In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pandas as pd
import librosa
import statistics 
import glob
import plotly.express as px
from pathlib import Path
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler  
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import PCA
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [13]:
# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "2gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [14]:
data_a = spark.read.csv('/content/files/ss13pusa.csv',header=True, inferSchema=True)#load the data and get a row (this is a dataframe)
data_b = spark.read.csv('/content/files/ss13pusb.csv',header=True, inferSchema=True) 

data = data_a.union(data_b)

data.show(2)

+---+--------+-------+----+---+-------+-----+----+---+-----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+----+-----+-----+----+----+---+-----+-----+-----+-----+------+---+---+----+----+-----+----+-----+----+----+----+----+----+----+----+----+----+---+---+----+----+---+----+----+----+---+----+---+-----+----+---+----+---+----+---+-----+-----+------+---+-------+----+---+-----+-----+-----+----+----+----+----+----+-------+-----+---+------+--------+----+---+----+----+-----+-----+----+------+-------+-----+-------+------+------+-----+-----+-----+-------+------+------+-----+------+-----+------+------+---+-------+---------+----+----+------+----+----+-----+-----+-----+------+-----+------+------+------+-----+------+------+------+-------+------+-----+-----+-----+-----+-----+-----+-----+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-----+-----+-----+-----+------+------+------+-----+------+-------+--

In [15]:
#transforming all the dataframe columns to int for correctly fill Nan and null value

from pyspark.sql.types import IntegerType
for col in data.columns:
  data = data.withColumn(col,data[col].cast(IntegerType()))

data = data.na.fill(0) 

In [16]:
#removing the PERNP column
new_col = data.columns
new_col.remove('PERNP')
new_col.remove('PINCP')




vectorAssembler = VectorAssembler(inputCols = new_col,    
                                  outputCol = 'features') 


df_full = vectorAssembler.transform(data)
df_full = df_full.select(['features', 'PINCP'])
df_full.show(5)

+--------------------+-----+
|            features|PINCP|
+--------------------+-----+
|(281,[1,2,3,4,5,6...|    0|
|(281,[1,2,3,4,5,6...|52000|
|(281,[1,2,3,4,5,6...|99000|
|(281,[1,2,3,4,5,6...|    0|
|(281,[1,2,3,4,5,6...|    0|
+--------------------+-----+
only showing top 5 rows



In [17]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

pca = PCA(k=20, 
          inputCol="features", 
          outputCol="pcaFeatures")

model = pca.fit(df_full)

result = model.transform(df_full)
result.show(5)

+--------------------+-----+--------------------+
|            features|PINCP|         pcaFeatures|
+--------------------+-----+--------------------+
|(281,[1,2,3,4,5,6...|    0|[-84.065349464680...|
|(281,[1,2,3,4,5,6...|52000|[-239.98024490026...|
|(281,[1,2,3,4,5,6...|99000|[-260.53340347800...|
|(281,[1,2,3,4,5,6...|    0|[-154.06172172305...|
|(281,[1,2,3,4,5,6...|    0|[-154.06242385407...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [18]:
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import MaxAbsScaler
from pyspark.sql.functions import col
# train test split 
split_df_pp = result.randomSplit([0.75, 0.25])
df_train = split_df_pp[0]
df_test = split_df_pp[1]

#here we can put a good condition on PINCP > 0 


##Scaling DenseVector (input)

#maxAbs scaler
scaler = MaxAbsScaler(inputCol="pcaFeatures", outputCol="scaled_pca_Features")
scalerModel = scaler.fit(df_train)
#scaled train
df_pp_train = scalerModel.transform(df_train)
#scaled test
df_pp_test = scalerModel.transform(df_test) #it is transformed in accordance with range of train



##Scaling Target variable (PINCP)

min_ = df_pp_train.agg({"PINCP": "min"}).collect()[0][0]
max_ = df_pp_train.agg({"PINCP": "max"}).collect()[0][0]

#scaled train
df_pp_train = df_pp_train.withColumn('scaled_PINCP', (col('PINCP') - min_ )/ max_)
#scaled test
df_pp_test = df_pp_test.withColumn('scaled_PINCP', (col('PINCP')-min_ )/ max_) #it is transformed in accordance with range of train


df_pp_train.show(2)

+--------------------+-----+--------------------+--------------------+-------------------+
|            features|PINCP|         pcaFeatures| scaled_pca_Features|       scaled_PINCP|
+--------------------+-----+--------------------+--------------------+-------------------+
|(281,[1,2,3,4,5,6...|76400|[-1346063.6211065...|[-0.9015539176260...|0.06947697111631537|
|(281,[1,2,3,4,5,6...|45880|[-577225.43648477...|[-0.3866086605835...|0.04565183450429352|
+--------------------+-----+--------------------+--------------------+-------------------+
only showing top 2 rows



In [19]:
## boost it! scaled
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol = 'scaled_pca_Features', 
                   labelCol = 'scaled_PINCP',
                   maxIter=10,
                   #maxDepth = ,
                   #stepSize = ,
                   seed = 123,
                   )

#fitting the model on train data
gbt_model = gbt.fit(df_pp_train)

#prediction on test data
gbt_predictions = gbt_model.transform(df_pp_test)

gbt_predictions.select('prediction','scaled_PINCP', 'PINCP', 'features').show(5)

+--------------------+--------------------+-----+--------------------+
|          prediction|        scaled_PINCP|PINCP|            features|
+--------------------+--------------------+-----+--------------------+
| 0.03296195224557169|0.035597189695550355|33000|(281,[1,2,3,4,5,6...|
| 0.03903994302887487| 0.04457455113192818|44500|(281,[1,2,3,4,5,6...|
| 0.02708350396685353|0.027478532396565183|22600|(281,[1,2,3,4,5,6...|
| 0.06565301875172624| 0.06760343481654957|74000|(281,[1,2,3,4,5,6...|
|0.024754881410869792|0.023887587822014052|18000|(281,[1,2,3,4,5,6...|
+--------------------+--------------------+-----+--------------------+
only showing top 5 rows



In [20]:
from pyspark.ml.evaluation import RegressionEvaluator
gbt_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)

gbt_evaluator = RegressionEvaluator(
    labelCol="scaled_PINCP", predictionCol="prediction", metricName="r2")
r2 = gbt_evaluator.evaluate(gbt_predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("R Squared (R2) on test data = %g" % r2)

Root Mean Squared Error (RMSE) on test data = 0.0103397
R Squared (R2) on test data = 0.932374
